# <center>Hidden Markov Model</center>

In [1]:
import numpy as np

In [50]:
# State Transition Matrix
A = np.array([[0.1,0.5,0.4],    
              [0.6,0.3,0.1],
              [0.3,0.6,0.1]])

# State-Output Association Probablity Matrix
B = np.array([[0.3,0.4,0.3],
              [0.4,0.2,0.4],
              [0.6,0.2,0.2]])

# Initial Probablity Vector
p = np.array([0.3,0.4,0.3])

# Observable State Vector
O = [1,0,2,2,1,0] 

In [51]:
def forward_var(A,B,p,O):
    T = len(O) 
    N = A.shape[0]
    alpha = np.zeros((T,N))
    for t in range(alpha.shape[0]):
        for i in range(alpha.shape[1]):
            if t==0:
                alpha[t,i] = B[i,O[0]]*p[i]
            else:
                sum1=0
                for k in range(A.shape[0]):
                    sum1 += alpha[t-1,k]*A[k,i]    
                alpha[t,i] =  sum1*B[i,O[t]]
    return(alpha)

In [52]:
alpha = forward_var(A,B,p,O)
print(alpha)

[[0.12       0.08       0.06      ]
 [0.0234     0.048      0.0372    ]
 [0.01269    0.019368   0.003576  ]
 [0.00418878 0.0057204  0.00147408]
 [0.00171734 0.00093899 0.00047899]
 [0.00026365 0.0005711  0.00049724]]


In [53]:
def backward_var(A,B,p,O):
    T = len(O)
    N = A.shape[0]
    beta = np.zeros((T,N))
    for t in range(beta.shape[0]-1,-1,-1):
        for i in range(beta.shape[1]):
            if t == beta.shape[0]-1:
                beta[t,i] = 1
            else:
                sum1=0
                for j in range(A.shape[0]):
                    sum1 += A[i,j]*B[j,O[t+1]]*beta[t+1,j]    
                beta[t,i] =  sum1
    return(beta)

In [54]:
beta = backward_var(A,B,p,O)
print(beta)

[[0.0058493  0.00432376 0.00473626]
 [0.01164804 0.01217352 0.01277148]
 [0.039612   0.034692   0.044016  ]
 [0.086      0.1422     0.1074    ]
 [0.47       0.36       0.39      ]
 [1.         1.         1.        ]]


In [55]:
def forward_eval(alpha):
    prob = np.sum(alpha[-1,:])
    return(prob)

In [56]:
forward_prob = forward_eval(alpha)
print(forward_prob)

0.001331992152


In [57]:
def backward_eval(B,beta,O):
    vec1 = np.ravel(B[:,O[0]])
    vec2 = np.ravel(beta[0,:])
    prob = np.dot(vec1,vec2)
    return(prob)

In [58]:
backward_prob = backward_eval(B,beta,O)
print(backward_prob)

0.00415172328


In [59]:
def gammaEval(alpha,beta):
    gamma = np.multiply(alpha,beta) ## Element-wise product
    statewise_sum = np.sum(gamma,axis=1)
    statewise_sum = statewise_sum.reshape(len(statewise_sum),1)
    gamma = gamma/statewise_sum
    return(gamma)

In [60]:
gamma = gammaEval(alpha,beta)
print(gamma)

[[0.5269671  0.25968672 0.21334618]
 [0.20462894 0.43868799 0.35668307]
 [0.37738682 0.5044434  0.11816978]
 [0.27044835 0.61069495 0.1188567 ]
 [0.60597076 0.25378301 0.14024623]
 [0.197935   0.42875961 0.37330539]]


In [61]:
def Viterbi(A,B,p,O):
    T = len(O)
    N = A.shape[0]
    delta = np.zeros((T,N))
    psi = np.zeros((T,N))
    optim_state = []
    for t in range(delta.shape[0]):
        for i in range(delta.shape[1]):
            if t==0:
                delta[t,i] = B[i,O[0]]*p[i]
                psi[t,i] = 0
            else:
                ls = []
                for k in range(N):
                    val = delta[t-1,k]*A[k,i]*B[i,O[t]]
                    ls.append(val)
                arr = np.array(ls)
                max_val = np.max(arr)
                max_val_idx = np.argmax(arr)
                delta[t,i] = max_val
                psi[t,i] = max_val_idx
    final_state_val = np.ravel(delta[-1,:])
    idx = np.argmax(final_state_val)
    optim_state.append(idx)
    for t in range(T-2,-1,-1):
        idx = psi[t,idx]
        idx = int(idx)
        optim_state.append(idx)
    optim_state = optim_state[::-1] 
    return(optim_state)

In [62]:
optim_state = Viterbi(A,B,p,O)
print(optim_state)

[0, 0, 2, 1, 0, 2]


In [63]:
def joint_state_measure(A,B,alpha,beta,O):
    N = A.shape[0]
    T = len(O)
    eta = np.zeros((T-1,N,N))
    
    for t in range(eta.shape[0]):
        sum1 = 0
        for i in range(eta.shape[1]):
            for j in range(eta.shape[2]):
                eta[t,i,j] = alpha[t,i]*A[i,j]*B[j,O[t+1]]*beta[t+1,j]
                sum1 += eta[t,i,j]
        eta[t,:,:] = eta[t,:,:]/sum1
    return(eta)

In [64]:
eta = joint_state_measure(A,B,alpha,beta,O)
print(eta)

[[[0.03148137 0.219344   0.27614174]
  [0.1259255  0.0877376  0.04602362]
  [0.04722206 0.1316064  0.03451772]]

 [[0.02087672 0.12189153 0.06186069]
  [0.25694422 0.15002034 0.03172343]
  [0.09956588 0.23253153 0.02458566]]

 [[0.02457987 0.27095025 0.0818567 ]
  [0.22508889 0.24812125 0.03123327]
  [0.02077958 0.09162346 0.00576674]]

 [[0.05912127 0.11321094 0.09811615]
  [0.48443312 0.09276379 0.03349804]
  [0.06241637 0.04780828 0.00863205]]

 [[0.03867898 0.2578599  0.30943188]
  [0.1268915  0.08459434 0.04229717]
  [0.03236452 0.08630537 0.02157634]]]


In [65]:
def Baum_Welch(eta,gamma,A,B,p,O):
    N = A.shape[0]
    M = B.shape[1]
    mod_A = np.zeros((N,N))
    mod_B = np.zeros((N,M))
    mod_p = np.zeros(N)
    
    for i in range(len(mod_p)):
        mod_p[i] = gamma[0,i]
    
    for i in range(mod_A.shape[0]):
        for j in range(mod_A.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(eta.shape[0]):
                num_sum += eta[t,i,j]
                denom_sum += gamma[t,i]
            a = num_sum/denom_sum
            mod_A[i,j] = a 
    
    for j in range(B.shape[0]):
        for k in range(B.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(gamma.shape[0]):
                if O[t] == k:
                    num_sum += gamma[t,j]
                denom_sum += gamma[t,j]
            b = num_sum/denom_sum
            mod_B[j,k] = b
            
    return(mod_A,mod_B,mod_p)

In [66]:
Baum_Welch(eta,gamma,A,B,p,O)

(array([[0.08801151, 0.49524309, 0.4167454 ],
        [0.58979614, 0.32082357, 0.08938029],
        [0.27694275, 0.62268956, 0.10036769]]),
 array([[0.18438012, 0.51890197, 0.2967179 ],
        [0.34752734, 0.20571245, 0.44676021],
        [0.55276723, 0.26774984, 0.17948293]]),
 array([0.5269671 , 0.25968672, 0.21334618]))